In [109]:
import numpy as np
import pandas as pd
import math
import sys

from sklearn.datasets import make_classification
from sklearn.datasets import make_regression

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

In [79]:
class DecisionNode:
    """
    Class for a parent/leaf node in the decision tree.
    A Node with node information about it's left and right nodes if any. it has the impurity info also.
    """
    def __init__(self, impurity=None, question=None, feature_index=None, threshold=None,
                 true_subtree=None, false_subtree=None):
        """
        :param
        """
        self.impurity = impurity
        # Which question to ask , to split the dataset.
        self.question = question 
        # Index of the feature which make the best fit for this node.
        self.feature_index = feature_index
        # The threshold value for that feature to make the split.
        self.threshold = threshold
        # DecisionNode Object of the left subtree.
        self.true_left_subtree = true_subtree
        # DecisionNode Object of the right subtree.
        self.false_right_subtree = false_subtree

class LeafNode:
    """ Leaf Node of the decision tree."""
    def __init__(self, value):
        self.prediction_value = value

In [ ]:
class DecisionTree:
    """Common class for making decision tree for classification and regression tasks."""
    def __init__(self, min_sample_split=3, min_impurity=1e-7, max_depth=float('inf'),
                 impurity_function=None, leaf_node_calculation=None):
        """
        """
        self.root = None

        self.min_sample_split = min_sample_split
        self.min_impurity = min_impurity
        self.max_depth = max_depth
        self.impurity_function = impurity_function
        self.leaf_node_calculation = leaf_node_calculation

    def _partition_dataset(self, Xy, feature_index, threshold):
        """Split the dataset based on the given feature and threshold.
        
        """
        col = Xy[:, feature_index]

        if isinstance(threshold, (int, float)):
            mask = col >= threshold
        else:
            mask = (col == threshold)

        # vectorised split
        X_1 = Xy[mask]
        X_2 = Xy[~mask]

        return X_1, X_2

    def _find_best_split(self, Xy):
        """ Find the best question/best feature threshold which splits the data well.
        
        """
        best_question = tuple() # this will containe the feature and its value which make the best split(higest gain).
        best_datasplit = {} # best data split.
        largest_impurity = 0
        n_features = (Xy.shape[1] - 1)
        # iterate over all the features.
        for feature_index in range(n_features):
            # find the unique values in that feature.
            col = Xy[:, feature_index]
            unique_value = np.unique(col)
            
            # iterate over all the unique values to find the impurity.
            for threshold in unique_value:
                # split the dataset based on the feature value.
                true_xy, false_xy = self._partition_dataset(Xy, feature_index, threshold)
                # skip the node which has any on type 0. because this means it is already pure.
                if len(true_xy) > 0 and len(false_xy) > 0:
                    

                    # find the y values.
                    y = Xy[:, -1]
                    true_y = true_xy[:, -1]
                    false_y = false_xy[:, -1]

                    # calculate the impurity function.
                    impurity = self.impurity_function(y, true_y, false_y)

                    # if the calculated impurity is larger than save this value for comaparition.
                    if impurity > largest_impurity:
                        largest_impurity = impurity
                        best_question = (feature_index, threshold)
                        best_datasplit = {
                                    "leftX": true_xy[:, :n_features],   # X of left subtree
                                    "lefty": true_xy[:, n_features:],   # y of left subtree
                                    "rightX": false_xy[:, :n_features],  # X of right subtree
                                    "righty": false_xy[:, n_features:]   # y of right subtree
                        }
                    
        return largest_impurity, best_question, best_datasplit

    def _build_tree(self, X, y, current_depth=0):
        """
        This is a recursive method to build the decision tree.
        """
        n_samples , n_features = X.shape
        # Add y as last column of X
        Xy = np.concatenate((X, y), axis=1)
        # find the Information gain on each feature each values and return the question which splits the data very well
        # based on the impurity function. (classfication - Information gain, regression - variance reduction).
        if (n_samples >= self.min_sample_split) and (current_depth <= self.max_depth):
            # find the best split/ which question split the data well.
            impurity, quesion, best_datasplit = self._find_best_split(Xy)
            if impurity > self.min_impurity:
            # Build subtrees for the right and left branch.
                true_branch = self._build_tree(best_datasplit["leftX"], best_datasplit["lefty"], current_depth + 1)
                false_branch = self._build_tree(best_datasplit["rightX"], best_datasplit["righty"], current_depth + 1)
                return DecisionNode( impurity=impurity, question=quesion, feature_index=quesion[0], threshold=quesion[1],
                                    true_subtree=true_branch, false_subtree=false_branch)

        leaf_value = self._leaf_value_calculation(y)
        return LeafNode(value=leaf_value)


    def train(self, X, y):
        """
        Build the decision tree.

        :param X: Train features/dependant values.
        :param y: train target/independant value.
        """
        self.root = self._build_tree(X, y, current_depth=0)

    def predict_sample(self, x, tree=None):
        """move form the top to bottom of the tree make a prediction of the sample by the
            value in the leaf node """
        if tree is None:
            tree = self.root
        # if it a leaf node the return the prediction.
        if isinstance(tree , LeafNode):

            return tree.prediction_value
        feature_value = x[tree.feature_index]

        branch = tree.false_right_subtree

        if isinstance(feature_value, int) or isinstance(feature_value, float):
            
            if feature_value >= tree.threshold:

                branch = tree.true_left_subtree
        elif feature_value == tree.threshold:
            branch = tree.true_left_subtree

        return self.predict_sample(x, branch)

    def predict(self, test_X):
        """ predict the unknow feature."""
        x = np.array(test_X)
        y_pred = [self.predict_sample(sample) for sample in x]
        # y_pred = np.array(y_pred)
        # y_pred = np.expand_dims(y_pred, axis = 1)
        return y_pred
    
    def draw_tree(self, tree = None, indentation = " "):
        """print the whole decitions of the tree from top to bottom."""
        if tree is None:
            tree = self.root

        def print_question(question, indention):
            """
            :param question: tuple of feature_index and threshold.
            """
            feature_index = question[0]
            threshold = question[1]

            condition = "=="
            if isinstance(threshold, int) or isinstance(threshold, float):
                condition = ">="
            print(indention,"Is {col}{condition}{value}?".format(col=feature_index, condition=condition, value=threshold))

        if isinstance(tree , LeafNode):
            print(indentation,"The predicted value -->", tree.prediction_value)
            return
        
        else:
            # print the question.
            print_question(tree.question,indentation)
            if tree.true_left_subtree is not None:
                # travers to the true left branch.
                print (indentation + '----- True branch :)')
                self.draw_tree(tree.true_left_subtree, indentation + "  ")
            if tree.false_right_subtree is not None:
                # travers to the false right-side branch.
                print (indentation + '----- False branch :)')
                self.draw_tree(tree.false_right_subtree, indentation + "  ")


  

In [131]:
class DecisionTreeClassifier(DecisionTree):
    """ Decision Tree for the classification problem."""
    def __init__(self, min_sample_split=3, min_impurity=1e-7, max_depth=float('inf'),
                 ):
        """
        :param min_sample_split: min value a leaf node must have.
        :param min_impurity: minimum impurity.
        :param max_depth: maximum depth of the tree.
        """
        self._impurity_function = self._claculate_information_gain
        self._leaf_value_calculation = self._calculate_majarity_class
        super(DecisionTreeClassifier, self).__init__(min_sample_split=min_sample_split, min_impurity=min_impurity, max_depth=max_depth,
                         impurity_function=self._impurity_function, leaf_node_calculation=self._leaf_value_calculation)
    
    def _entropy(self, y):
        """ Find the entropy for the given data"""
        entropy = 0
        unique_value = np.unique(y)
        for val in unique_value:
            # probability of that class.
            p = len(y[y==val]) / len(y)
            entropy += -p * (math.log(p) / math.log(2))
        return entropy


    def _claculate_information_gain(self, y, y1, y2):
        """
        Calculate the information gain.

        :param y: target value.
        :param y1: target value for dataset in the true split/right branch.
        :param y2: target value for dataset in the false split/left branch.
        """
        # propobility of true values.
        p = len(y1) / len(y)
        entropy = self._entropy(y)
        info_gain = entropy - p * self._entropy(y1) - (1 - p) * self._entropy(y2)
        return info_gain       

    def _calculate_majarity_class(self, y):
        """
        calculate the prediction value for that leaf node.
        
        :param y: leaf node target array.
        """
        most_frequent_label = None
        max_count = 0
        unique_labels = np.unique(y)
        # iterate over all the unique values and find their frequentcy count.
        for label in unique_labels:
            count = len( y[y == label])
            if count > max_count:
                most_frequent_label = label
                max_count = count
        return most_frequent_label

    def train(self, X, y):
        """
        Build the tree.

        :param X: Feature array/depentant values.
        :parma y: target array/indepentant values.
        """
        # train the model.
        super(DecisionTreeClassifier, self).train(X, y)
    
    def predict(self, test_X):
        """ predict the unknow feature."""
        y_pred = super(DecisionTreeClassifier, self).predict(test_X)
        y_pred = np.array(y_pred)
        y_pred = np.expand_dims(y_pred, axis = 1)
        return y_pred
    
    

# Data Creation.

In [132]:
# X = np.array([["Green", 3], ["yello", 3], ["orange_color", 2], ["orange_color", 2], ["red", 1]])
# y = np.array(["apply", "apply", "orange", "orange", "cherry"])
# X = pd.DataFrame(X)
# y = pd.DataFrame(y)
# y.head

# Define the traning data.
X, y = make_classification(n_samples=1000, n_classes=2, n_features=5)

# Chnage the shape of the target to 1 dimentional array.
y = y[:, np.newaxis]

print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))
print("Shape of the target value ----------> {}".format(y.shape))

Number of training data samples-----> 1000
Number of training features --------> 5
Shape of the target value ----------> (1000, 1)


In [133]:
# display the data.
data = pd.DataFrame(X)
data.head()

,0,1,2,3,4
0,-1.829463,1.017113,-0.532986,0.803406,1.450504
1,1.142740,0.002981,0.616051,-0.067555,-0.643803
2,1.046571,-0.041782,1.374942,1.181671,0.161260
3,-0.253063,0.965875,-1.118723,-1.491727,-0.767204
4,1.921559,1.121124,1.665487,0.852074,-0.499481


In [134]:
# display the data.
data_y = pd.DataFrame(y)
data_y.head()

,0
0,1
1,0
2,1
3,0
4,1


In [135]:
#define the parameters
sys.setrecursionlimit(2000)
param = {
    "n_neibours" : 5
}
print("="*100)
decirion_tree_cla = DecisionTreeClassifier(min_sample_split=2, max_depth=45)

# Train the model.
decirion_tree_cla.train(X, y) 
# print the decision tree.
print("Printing the tree :).....")
decirion_tree_cla.draw_tree()
# Predict the values.
y_pred = decirion_tree_cla.predict(X)

#calculate accuracy.
acc = np.sum(y==y_pred)/X.shape[0]
print("="*100)
print("Accuracy of the prediction is {}".format(acc))

Printing the tree :).....
  Is 3>=0.2158312001199094?
 ----- True branch :)
    Is 3>=1.603491216736123?
   ----- True branch :)
      The predicted value --> 1.0
   ----- False branch :)
      Is 3>=1.6027001267926013?
     ----- True branch :)
        The predicted value --> 0.0
     ----- False branch :)
        Is 1>=1.1102558720057867?
       ----- True branch :)
          Is 1>=1.121124484926718?
         ----- True branch :)
            The predicted value --> 1.0
         ----- False branch :)
            The predicted value --> 0.0
       ----- False branch :)
          The predicted value --> 1.0
 ----- False branch :)
    Is 0>=-1.4505020514700175?
   ----- True branch :)
      Is 0>=-1.2409267655158718?
     ----- True branch :)
        Is 4>=-0.9855838987198552?
       ----- True branch :)
          The predicted value --> 0.0
       ----- False branch :)
          Is 4>=-0.9919443440822534?
         ----- True branch :)
            Is 0>=-0.03912201395755674?
           -

# Decision tree classifier using scikit-learn for comaprision.

In [106]:
from sklearn.tree import DecisionTreeClassifier as DecisionTreeClassifier_sklearn

In [107]:
# data is already defined, going to use the same data for comparision.
print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))

Number of training data samples-----> 1000
Number of training features --------> 5


In [110]:
decision_tree_sklearn = DecisionTreeClassifier_sklearn()
decision_tree_sklearn.fit(X, y)

# predict the value
y_pred_sklearn = decision_tree_sklearn.predict(X)
acc = accuracy_score(y, y_pred_sklearn)
print("="*100)
print("Accuracy of the prediction is {}".format(acc))

Accuracy of the prediction is 1.0


# Decision Tree Regression.

In [119]:
class DecisionTreeRegression(DecisionTree):
    """ Decision Tree for the classification problem."""
    def __init__(self, min_sample_split=3, min_impurity=1e-7, max_depth=float('inf'),
                 ):
        """
        :param min_sample_split: min value a leaf node must have.
        :param min_impurity: minimum impurity.
        :param max_depth: maximum depth of the tree.
        """
        self._impurity_function = self._claculate_variance_reduction
        self._leaf_value_calculation = self._calculate_colum_mean
        super(DecisionTreeRegression, self).__init__(min_sample_split=min_sample_split, min_impurity=min_impurity, max_depth=max_depth,
                         impurity_function=self._impurity_function, leaf_node_calculation=self._leaf_value_calculation)
    

    def _claculate_variance_reduction(self, y, y1, y2):
        """
        Calculate the Variance reduction.

        :param y: target value.
        :param y1: target value for dataset in the true split/right branch.
        :param y2: target value for dataset in the false split/left branch.
        """
        # propobility of true values.
        variance = np.var(y)
        variance_y1 = np.var(y1)
        variance_y2 = np.var(y2)

        y_len = len(y)
        fraction_1 = len(y1) / y_len 
        fraction_2 = len(y2) / y_len
        variance_reduction = variance - (fraction_1 * variance_y1 + fraction_2 * variance_y2)
        return  variance_reduction

    def _calculate_colum_mean(self, y):
        """
        calculate the prediction value for that leaf node using mean.
        
        :param y: leaf node target array.
        """
        mean = np.mean(y, axis=0)
        return mean

    def train(self, X, y):
        """
        Build the tree.

        :param X: Feature array/depentant values.
        :parma y: target array/indepentant values.
        """
        # train the model.
        super(DecisionTreeRegression, self).train(X, y)
    
    

# create the data.

In [123]:
# Define the traning data.
X, y = make_regression(n_samples=1000, n_features=8)

# Chnage the shape of the target to 1 dimentional array.
y = y[:, np.newaxis]

print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))
print("Shape of the target value ----------> {}".format(y.shape))

Number of training data samples-----> 1000
Number of training features --------> 8
Shape of the target value ----------> (1000, 1)


In [124]:
# display the data.
data_y = pd.DataFrame(y)
data_y.head()

,0
0,-89.458799
1,-27.869511
2,-94.535230
3,-17.165601
4,-311.382545


In [126]:
#define the parameters
sys.setrecursionlimit(2000)

print("="*100)
decirion_tree_reg = DecisionTreeRegression(min_sample_split=2, max_depth=45)

# Train the model.
decirion_tree_reg.train(X, y) 
print("Printing the mdoel :)...")
decirion_tree_reg.draw_tree()
# Predict the values.
y_pred = decirion_tree_reg.predict(X)
#Root mean square error.
score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

Printing the mdoel :)...
  Is 0>=-0.21438011135558163?
 ----- True branch :)
    Is 1>=0.7907654865740426?
   ----- True branch :)
      Is 4>=-0.5439254515196377?
     ----- True branch :)
        Is 0>=1.1736031114466987?
       ----- True branch :)
          Is 4>=1.2250522851779126?
         ----- True branch :)
            Is 6>=0.559415815161355?
           ----- True branch :)
              Is 0>=2.039182631837753?
             ----- True branch :)
                The predicted value --> [498.04718411]
             ----- False branch :)
                Is 0>=1.6624438066775729?
               ----- True branch :)
                  The predicted value --> [428.02792105]
               ----- False branch :)
                  The predicted value --> [442.20262889]
           ----- False branch :)
              Is 0>=2.021069263910125?
             ----- True branch :)
                Is 0>=2.0240234468149447?
               ----- True branch :)
                  The predicted value

# Decision tree Regression using scikit learn for comparision.

In [137]:
from sklearn.tree import DecisionTreeRegressor as DecisionTreeRegressor_sklearn

In [138]:
# data is already defined, going to use the same data for comparision.
print("="*100)
print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))

Number of training data samples-----> 1000
Number of training features --------> 5


In [139]:
decision_tree_reg_sklearn = DecisionTreeRegressor_sklearn()
decision_tree_reg_sklearn.fit(X, y)

# predict the value
y_pred_sklearn = decision_tree_reg_sklearn.predict(X)
socre = r2_score(y, y_pred_sklearn)
print("="*100)
print("r2_score of the prediction is {}".format(socre))

r2_score of the prediction is 1.0


# Supervised Machine Learning models scratch series....
you can also check....

- 1) Linear Regression         ---> https://www.kaggle.com/ninjaac/linear-regression-from-scratch
- 2) Lasso Regression          ---> https://www.kaggle.com/ninjaac/lasso-and-ridge-regression-from-scratch 
- 3) Ridge Regression          ---> https://www.kaggle.com/ninjaac/lasso-and-ridge-regression-from-scratch
- 4) ElasticNet Regression     ---> https://www.kaggle.com/ninjaac/elasticnet-regression-from-scratch 
- 5) Polynomail Regression     ---> https://www.kaggle.com/ninjaac/polynomial-and-polynomialridge-regression-scratch 
- 5) PolynomailRidge Regression---> https://www.kaggle.com/ninjaac/polynomial-and-polynomialridge-regression-scratch 
- 6) KNN Classifier            ---> https://www.kaggle.com/ninjaac/knnclassifier-from-scratch 
- 7) Decision Tree Classifier  ---> (Same Notebook you are looking now)